In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from itertools import chain
from sys import platform
from copy import deepcopy as dc

vims_wave = np.loadtxt('vims_wave.txt')

new_array = ['_0405','_0607','_0809','_1011','_1213','_1415','_1617']
if platform == 'darwin' or platform == 'win32':
    for number in new_array:
        new_string = 'df'+number+' = pd.read_pickle("e_data/data'+number+'.pkl")'
        exec(new_string)
else:
    df_list = []
    for number in new_array:
        new_string = 'df'+number+' = pd.read_csv("e_data/data'+number+'.csv")'
        exec(new_string)
        df = 'df'+number
        new_df = locals()[df]
        df_list.append(new_df)
    for df in df_list:
        for i in range(len(df['spectrum'])):
            df['spectrum'][i] = np.array([float(x) for x in df['spectrum'][i][1:-1].split(',')])

def powerlaw(x,a,b):
    return a*np.power(x,b)

def gaussian(x,a,mu,sigma):
    return a*np.exp(-(x-mu)**2/(2*sigma**2))

def flatten(some_list):
    flat_list = []
    for element in some_list:
        if type(element) is list:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

band_channels = list(chain(range(29,35),range(46,60),range(78,96),range(102,106)))

window = list(range(56,83))

def fit_line(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_fit = powerlaw(vims_wave,*my_fit)
    return final_fit

def custom_fit(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_spectra = spectra-powerlaw(vims_wave,*my_fit)
    return final_spectra

def single_fit(spectrum):
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectrum[band_channels],p0=[.15,-12])
    final_spectrum = spectrum-powerlaw(vims_wave,*my_fit)

def shift_ret(spectrum):
    my_fit,_ = curve_fit(gaussian,vims_wave[window],spectrum[window],p0=[.05,2.05])
    return my_fit[1]

In [2]:
time_list_i = []
for number in new_array:
    my_string = 'ettime'+number+' = df'+number+"['ettime']"
    exec(my_string)
    time = 'ettime'+number
    time_list_i.append(locals()[time])

time_list_f = []
for df in time_list_i:
    for time in df:
        time_list_f.append(time)


spec_list = []
for number in new_array:
    my_string = 'old_spec'+number+' = df'+number+"['spectrum']"
    exec(my_string)
    new_string = 'spec'+number+' = np.array([np.abs(x) for x in old_spec'+number+'])'
    exec(new_string)
    spec = 'spec'+number
    spec_list.append(locals()[spec])

copy_list = []
for number in new_array:
    my_string = 'copy_spec'+number+' = dc(spec'+number+')'
    exec(my_string)
    copy_spec = 'copy_spec'+number
    copy_list.append(locals()[copy_spec])

total_spectrum = []
for spec in spec_list:
    for spectrum in spec:
        total_spectrum.append(spectrum)

total_spectrum = np.asarray(total_spectrum)
total_spectrum_2 = dc(total_spectrum)
print(np.shape(total_spectrum))
print(np.shape(total_spectrum_2))


(12655, 256)
(12655, 256)


In [3]:
changed_list = []
for (spec,copy_spec) in zip(spec_list,copy_list):
    channels = []
    for i in range(256):
        mean = np.mean(spec[:,i])
        vals = copy_spec[:,i]
        std = np.std(spec[:,i])
        ind = np.where(np.abs(vals-mean)>3*std)
        vals[ind] = mean
        changed_stuff = len(ind[0])
        channels.append(changed_stuff)
    
    for i in range(256):
        mean = np.mean(spec[:,i])
        vals = copy_spec[:,i]
        std = np.std(copy_spec[:,i])
        ind = np.where(np.abs(vals-mean)>3*std)
        vals[ind] = mean
        changed_stuff = len(ind[0])
        channels[i] += changed_stuff

    for i in range(256):
        mean = np.mean(copy_spec[:,i])
        vals = copy_spec[:,i]
        ind = np.where(np.abs(vals-0)<.01)
        vals[ind] = mean
        changed_stuff = len(ind[0])
        channels[i] += changed_stuff
    
    print(np.shape(channels))
    changed_list.append(channels)

(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)


#### **```changed_list``` array contains all information about each VIMS channel per spectra**

In [ ]:
%matplotlib inline
fig,axs = plt.subplots(3,3,figsize=(40,25),facecolor='white')
for index,spec in enumerate(copy_list):
    for spectrum in spec:
        if index < 3:
            axs[0,index].plot(vims_wave,spectrum)
        if index < 6 and index >= 3:
            axs[1,index-3].plot(vims_wave,spectrum)
        if index >= 6:
            axs[2,index-5].plot(vims_wave,spectrum)
    if index < 3:
        axs[0,index].grid()
    if index < 6 and index >= 3:
        axs[1,index-3].grid()
    if index >= 6:
        axs[2,index-5].grid()

plt.show()

In [ ]:
%matplotlib inline
plt.figure(facecolor='white',figsize=(12,8))
for spectrum in copy_spec_1415:
    plt.plot(vims_wave,spectrum)
plt.xlim([1.75,2.25])
plt.grid()
plt.show()